In [37]:
import pandas as pd
import time
import threading
from tqdm import tqdm
from binance.client import Client

client = Client()

### All tradable assets in this dictionnary

In [3]:
info = client.get_exchange_info()

In [9]:
symbols = [x['symbol'] for x in info['symbols']]

In [16]:
len(symbols)

2766

In [15]:
#exclude leveraged tokens
exclude = ['UP', 'DOWN', 'BEAR', 'BULL']
non_lev = [ symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]

In [14]:
len(non_lev)

2697

### Keep only usdt pair

In [17]:
relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')]

In [19]:
len(relevant)

471

In [20]:
klines = {}

### Get 1 minute candles for each symbol

In [21]:
for symbol in tqdm(relevant) : 
    klines[symbol] = client.get_historical_klines(symbol, '1m', '1 hour ago UTC')

100%|██████████| 471/471 [04:22<00:00,  1.80it/s]


In [25]:
(pd.DataFrame(klines['BTCUSDT'])[4].astype(float).pct_change() + 1 ).prod() -1 #4 column is the close

0.00021125056342108373

In [26]:
returns, symbols = [], []

for symbol in relevant:
    if len(klines[symbol]) > 0 : 
        cumret = (pd.DataFrame(klines[symbol])[4].astype(float).pct_change() + 1 ).prod() - 1
        returns.append(cumret)
        symbols.append(symbol)

In [27]:
retdf = pd.DataFrame(returns, index=symbols, columns=['ret'])

In [28]:
retdf.ret.nlargest(10)

API3USDT     0.043275
OAXUSDT      0.034763
ZECUSDT      0.024750
DYDXUSDT     0.016505
MOVRUSDT     0.015326
FTTUSDT      0.014781
RAYUSDT      0.012847
BTTCUSDT     0.011628
MAGICUSDT    0.010071
AGLDUSDT     0.008056
Name: ret, dtype: float64

### Function that return the top performers

In [34]:
import pandas as pd

def get_top_performer(client, interval, duration, n=10):
    info = client.get_exchange_info()
    symbols = [x['symbol'] for x in info['symbols']]
    
    #exclure les tokens à effet de levier
    exclude = ['UP', 'DOWN', 'BEAR', 'BULL']
    non_lev = [symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]
    relevant_symbols = [symbol for symbol in non_lev if symbol.endswith('USDT')]
    
    #récupérer les données de marché pour chaque symbole
    returns, selected_symbols = [], []
    for symbol in tqdm(relevant_symbols):
        klines = client.get_historical_klines(symbol=symbol, interval=interval, start_str=duration)
        
        if len(klines) > 0:
            #calcul du rendement cumulé
            df = pd.DataFrame(klines)
            df_close_prices = df[4].astype(float)  
            cumret = (df_close_prices.pct_change() + 1).prod() - 1
            
            returns.append(cumret)
            selected_symbols.append(symbol)
    
    #construire un DataFrame pour les rendements
    retdf = pd.DataFrame(returns, index=selected_symbols, columns=['ret'])
    return retdf.ret.nlargest(n)


In [35]:
top_performers = get_top_performer(client, '1m',  '10 minutes ago UTC', n=10)


100%|██████████| 471/471 [04:34<00:00,  1.72it/s]


In [36]:
top_performers

KP3RUSDT     0.026362
TROYUSDT     0.025681
APEUSDT      0.018535
RAYUSDT      0.008809
TURBOUSDT    0.008788
FTMUSDT      0.007834
RIFUSDT      0.006674
MEMEUSDT     0.005793
SKLUSDT      0.005366
OOKIUSDT     0.004673
Name: ret, dtype: float64

### Same with multithreading, 20 time faster

In [49]:
def get_history(client, symbol, interval, duration):
    time.sleep(0.01)  #délai pour contourner la limite de l'API
    return client.get_historical_klines(symbol=symbol, interval=interval, start_str=duration)

def get_top_performer_multithreading(client, interval, duration, n=10):
    #informations sur les symboles
    info = client.get_exchange_info()
    symbols = [x['symbol'] for x in info['symbols']]
    #exclure les tokens à effet de levier
    exclude = ['UP', 'DOWN', 'BEAR', 'BULL']
    non_lev = [symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]
    relevant_symbols = [symbol for symbol in non_lev if symbol.endswith('USDT')]

    #créer une liste pour stocker les données et une pour les rendements
    returns, selected_symbols = [], []
    klines_data = [None] * len(relevant_symbols)
    
    #fonction pour récupérer les données en parallèle
    def _helper(i):
        klines_data[i] = get_history(client, relevant_symbols[i], interval, duration)
    
    #créer et lancer les threads
    threads = [threading.Thread(target=_helper, args=(i,)) for i in range(len(relevant_symbols))]
    for thread in threads:
        thread.start()
    for thread in tqdm(threads):
        thread.join()
    
    #calcul des rendements
    for i, klines in enumerate(klines_data):
        if klines and len(klines) > 0:
            df = pd.DataFrame(klines)
            df_close_prices = df[4].astype(float)  # La colonne des prix de clôture
            cumret = (df_close_prices.pct_change() + 1).prod() - 1
            returns.append(cumret)
            selected_symbols.append(relevant_symbols[i])
    
    retdf = pd.DataFrame(returns, index=selected_symbols, columns=['ret'])
    return retdf.ret.nlargest(n)

In [47]:
top_performers = get_top_performer_multithreading(client, '1m',  '5 minutes ago UTC', n=10)

100%|██████████| 471/471 [00:08<00:00, 52.42it/s]


In [48]:
top_performers

BTTCUSDT     0.011494
CREAMUSDT    0.006479
PDAUSDT      0.004796
PROMUSDT     0.004271
SKLUSDT      0.003960
SYNUSDT      0.003949
DYMUSDT      0.003894
TURBOUSDT    0.003881
WOOUSDT      0.003753
AAVEUSDT     0.003401
Name: ret, dtype: float64